<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=37f5f0c341c6854cfffd25a18cae197dda471633c220ecf555ae51a3b154cd91
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 09:04:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 9.92 K (0.07%)
Current PnL: -17.62 L (-11.85%)
CY Booked + Current PnL: -6.59 L (-4.43%)
-------------------
Total profit:  2.12 L
Total loss:  -19.74 L
-------------------
Total Booked + Current PnL: 20.39 L (16.75%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.72 L (59.14%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.31,60.0,H-LC,11.18,173239.0,17062.0,9026.0,-0.06,10.92,5.21,16.71,22.0,1.89,1.26,30.27,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-11.34,61.0,M-LC,4.27,218241.0,16762.0,10454.0,-0.01,8.32,4.79,13.51,60.0,1.60,1.59,21.96,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,103.63,57.0,M-SC,4.99,87770.0,-13007.0,13104.0,-0.47,-12.91,14.93,0.09,245.0,-0.99,0.64,15.31,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.29,115699.0,15195.0,15446.0,1.82,15.12,13.35,30.49,79.0,0.98,0.84,52.42,MH,ATH,METALS
41,ITC,452.00,-37.50,55.0,H-LC,2.99,202324.0,2186.0,18695.0,-0.31,1.09,9.24,10.44,4.0,0.12,1.47,6.72,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,42.0,H-MC,2.07,135781.0,-7.0,42757.0,0.39,-0.00,31.49,31.48,107.0,-0.00,0.99,12.11,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-1.08,56.0,H-MC,1.67,245326.0,-2405.0,77474.0,-0.11,-0.97,31.58,30.30,92.0,-0.03,1.79,12.19,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180469.0,661.0,111746.0,-0.64,0.37,61.92,62.52,88.0,0.01,1.31,34.58,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299732.0,-16110.0,128765.0,-1.02,-5.10,42.96,35.67,5.0,-0.13,2.18,8.05,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.47,44.0,H-LC,13.52,267632.0,267.0,65195.0,0.16,0.10,24.36,24.48,48.0,0.00,1.95,55.13,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.05,225268.0,-8876.0,57488.0,-0.71,-3.79,25.52,20.77,86.0,-0.15,1.64,2.87,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,1.08,48.0,H-LC,10.11,230350.0,-11574.0,74841.0,-0.03,-4.78,32.49,26.15,8.0,-0.15,1.68,8.85,X40,ATH,IT
56,RELIANCE,1533.00,-12.91,53.0,H-LC,6.05,217620.0,7254.0,21523.0,0.85,3.45,9.89,13.68,37.0,0.34,1.58,20.54,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.18,60.0,H-LC,1.18,239962.0,-6189.0,85402.0,0.77,-2.51,35.59,32.18,16.0,-0.07,1.75,32.62,X200,ATH,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.99,202324.0,2186.0,18695.0,-0.31,1.09,9.24,10.44,4.0,0.12,1.47,6.72,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,3.91,284918.0,19492.0,37296.0,-0.26,7.34,13.09,21.40,11.0,0.52,2.07,15.70,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.09,56.0,H-LC,5.91,215720.0,11220.0,30201.0,0.88,5.49,14.00,20.25,10.0,0.37,1.57,14.70,X40N,BTT,PHARMA
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299732.0,-16110.0,128765.0,-1.02,-5.10,42.96,35.67,5.0,-0.13,2.18,8.05,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3872.97,65.0,M-SC,7.20,123829.0,-23539.0,75350.0,8.27,-15.97,60.85,35.16,236.0,-0.31,0.90,26.48,XY24,NTT,AUTO
32,HINDZINC,730.22,23.40,68.0,M-LC,1.69,201383.0,-3693.0,116258.0,3.74,-1.80,57.73,54.89,52.0,-0.03,1.47,21.86,X5K,ATH,METALS
75,TITAGARH,1548.00,-1.01,67.0,H-SC,5.76,162312.0,-29703.0,108587.0,2.95,-15.47,66.90,41.08,158.0,-0.27,1.18,37.54,XY24,NTT,ENGINEERING
48,MASFIN,398.61,-14.63,63.0,H-SC,13.18,96825.0,-1155.0,22754.0,2.17,-1.18,23.50,22.05,152.0,-0.05,0.70,40.74,XR,ATH,FINANCE
58,ROUTE,2236.84,-46.55,48.0,H-SC,18.04,88042.0,-45024.0,137883.0,1.85,-33.84,156.61,69.78,118.0,-0.33,0.64,2.80,SR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.0,242.38,70.0,M-SC,5.43,152939.0,-30139.0,109122.0,-2.72,-16.46,71.35,43.14,230.0,-0.28,1.11,43.86,XY24,NTT,MISC
67,SONACOMS,1006.0,-39.73,39.0,M-SC,12.03,74952.0,-26207.0,101095.0,-2.65,-25.91,134.88,74.03,202.0,-0.26,0.55,5.69,AR,BTT,AUTO
53,RAJESHEXPO,518.0,1821.72,61.0,L-SC,5.56,52446.0,-84731.0,84826.0,-2.26,-61.77,161.74,0.07,267.0,-1.00,0.38,30.44,OX40N,NTT,JEWELLERY
14,BERGEPAINT,680.0,-18.72,48.0,H-MC,6.57,218437.0,-8902.0,56968.0,-2.18,-3.92,26.08,21.14,106.0,-0.16,1.59,23.12,XY24,NTT,PAINTS
64,SHALBY,327.0,1054.70,66.0,M-SC,2.00,156964.0,-24179.0,69645.0,-1.77,-13.35,44.37,25.10,235.0,-0.35,1.14,25.79,XY24,NTT,HEALTHCARE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,99.53,60.0,H-SC,12.57,125874.0,-16605.0,36126.0,-1.31,-11.65,28.70,13.70,163.0,-0.46,0.92,48.57,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,103.63,57.0,M-SC,4.99,87770.0,-13007.0,13104.0,-0.47,-12.91,14.93,0.09,245.0,-0.99,0.64,15.31,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.29,44.0,H-SC,2.15,145406.0,-30497.0,72514.0,-0.31,-17.34,49.87,23.89,149.0,-0.42,1.06,25.58,OX40N,NTT,CERAMICS
44,KANSAINER,340.0,-67.61,59.0,H-SC,2.03,222786.0,-46881.0,83211.0,0.38,-17.38,37.35,13.47,138.0,-0.56,1.62,12.68,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.94,108708.0,-23127.0,65355.0,-0.54,-17.54,60.12,32.03,98.0,-0.35,0.79,21.76,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.40,294540.0,-51416.0,175281.0,0.29,-14.86,59.51,35.80,1.0,-0.29,2.14,4.34,X40,BTT,IT
39,INFY,2275.00,-16.73,56.0,H-LC,9.46,324953.0,12065.0,159617.0,1.05,3.86,49.12,54.87,3.0,0.08,2.37,10.76,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.99,202324.0,2186.0,18695.0,-0.31,1.09,9.24,10.44,4.0,0.12,1.47,6.72,X40,NTT,FMCG
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299732.0,-16110.0,128765.0,-1.02,-5.10,42.96,35.67,5.0,-0.13,2.18,8.05,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,56.0,H-LC,11.94,251933.0,-9686.0,128889.0,0.58,-3.70,51.16,45.57,7.0,-0.08,1.83,7.20,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,64.0,M-MC,6.95,227752.0,2790.0,164870.0,0.33,1.24,72.39,74.53,192.0,0.02,1.66,33.89,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.17,52.0,H-MC,3.95,188490.0,23820.0,44144.0,-0.10,14.47,23.42,41.28,89.0,0.54,1.37,23.60,X40N,ATH,AC
84,VOLTAS,1918.49,0.32,57.0,H-MC,2.97,210540.0,18798.0,77226.0,0.59,9.80,36.68,50.08,99.0,0.24,1.53,17.27,XY25,ATH,AC
24,DABUR,735.00,-3.48,56.0,H-MC,6.17,210152.0,13124.0,76495.0,-0.96,6.66,36.40,45.49,102.0,0.17,1.53,20.57,XY24,BTT,FMCG
18,CAMS,5211.76,-4.94,51.0,H-SC,4.73,108046.0,6042.0,37881.0,0.22,5.92,35.06,43.06,122.0,0.16,0.79,24.05,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.29,115699.0,15195.0,15446.0,1.82,15.12,13.35,30.49,79.0,0.98,0.84,52.42,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.40,101362.0,9864.0,68886.0,1.62,10.78,67.96,86.07,223.0,0.14,0.74,47.20,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.72,53.0,H-SC,12.24,137837.0,14501.0,119257.0,-0.45,11.76,86.52,108.45,119.0,0.12,1.00,36.74,AR,ATH,MISC
86,WIPRO,420.00,-11.07,54.0,M-LC,5.81,156203.0,5258.0,104203.0,-0.78,3.48,66.71,72.51,53.0,0.05,1.14,9.72,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180469.0,661.0,111746.0,-0.64,0.37,61.92,62.52,88.0,0.01,1.31,34.58,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.52,86880.0,-26669.0,66724.0,-0.92,-23.49,76.80,35.27,268.0,-0.40,0.63,112.64,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.19,42.0,H-SC,13.99,87316.0,-13647.0,108176.0,-0.95,-13.52,123.89,93.63,148.0,-0.13,0.64,30.05,SR,ATH,CHEMICALS
48,MASFIN,398.61,-14.63,63.0,H-SC,13.18,96825.0,-1155.0,22754.0,2.17,-1.18,23.50,22.05,152.0,-0.05,0.70,40.74,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.29,115699.0,15195.0,15446.0,1.82,15.12,13.35,30.49,79.0,0.98,0.84,52.42,MH,ATH,METALS
8,ATULAUTO,844.00,3872.97,65.0,M-SC,7.20,123829.0,-23539.0,75350.0,8.27,-15.97,60.85,35.16,236.0,-0.31,0.90,26.48,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.52,86880.0,-26669.0,66724.0,-0.92,-23.49,76.80,35.27,268.0,-0.40,0.63,112.64,XR,NTT,HOTELS
8,ATULAUTO,844.00,3872.97,65.0,M-SC,7.20,123829.0,-23539.0,75350.0,8.27,-15.97,60.85,35.16,236.0,-0.31,0.90,26.48,XY24,NTT,AUTO
49,NATIONALUM,247.44,-42.00,73.0,H-MC,4.29,115699.0,15195.0,15446.0,1.82,15.12,13.35,30.49,79.0,0.98,0.84,52.42,MH,ATH,METALS
75,TITAGARH,1548.00,-1.01,67.0,H-SC,5.76,162312.0,-29703.0,108587.0,2.95,-15.47,66.90,41.08,158.0,-0.27,1.18,37.54,XY24,NTT,ENGINEERING
60,SAMMAANCAP,326.00,-170.45,62.0,M-SC,3.01,81720.0,-20490.0,113877.0,-0.53,-20.05,139.35,91.37,208.0,-0.18,0.59,32.66,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,43.50
2,50,73.88


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.46
LC    34.55
MC    23.00
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.95
X40      15.29
X40N     12.96
XR       10.03
XY25      9.76
AR        8.60
OX40N     8.20
X200      1.75
MH        1.72
X5K       1.47
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.38
H-LC    26.29
H-MC    20.05
M-SC    13.53
M-LC     7.20
M-MC     2.64
L-SC     1.55
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.42,-14.12,79.00
FMCG,11.84,-4.02,40.41
FINANCE,9.62,-16.44,62.05
BANKS,7.79,-12.15,63.78
MISC,6.05,-14.00,79.08
PAINTS,5.87,-14.35,39.94
ELECTRICAL,5.60,-7.06,66.07
AC,3.61,2.41,30.63
INSURANCE,3.41,-7.47,44.33


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2866406.0,22
AR,1316238.0,9
XR,1234761.0,13
X40,866797.0,10
X40N,631985.0,10
OX40N,521386.0,10
XY25,408966.0,7
SR,246059.0,2
X5K,116258.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3354219.0,28
M-SC,1386297.0,17
H-LC,1380536.0,15
H-MC,1198269.0,15
M-LC,390985.0,5
M-MC,314929.0,2
L-SC,248405.0,3
L-MC,60391.0,1
L-LC,38345.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1037680.0      6
M-SC       XY24       798965.0      7
H-SC       AR         784904.0      5
           XR         678852.0      6
H-LC       X40        578362.0      5
H-MC       XY24       489567.0      4
H-LC       AR         367278.0      2
M-MC       XY24       314929.0      2
H-SC       X40N       296806.0      4
           OX40N      247246.0      4
           SR         246059.0      2
H-LC       X40N       225480.0      4
H-MC       X40        218985.0      4
           XY25       187471.0      2
M-SC       AR         164056.0      2
L-SC       XR         163579.0      2
M-LC       XY24       160070.0      2
M-SC       OX40N      123959.0      4
M-LC       X5K        116258.0      1
M-SC       XR         115990.0      2
           XY25       113877.0      1
H-MC       XR         111746.0      1
           X40N       109699.0      2
M-LC       XR         104203.0      1
H-LC       X200        85402.0      1
L-SC       OX40N       84826.0      1
M-SC       X40         69450.0      1
H-MC       OX40N       65355.0      1
H-LC       XY24        65195.0      1
H-SC       MH          62672.0      1
L-MC       XR          60391.0      1
H-LC       XY25        58819.0      2
L-LC       XY25        38345.0      1
H-MC       MH          15446.0      1
M-LC       XY25        10454.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
